In [ ]:
# 🔧 Instalación de librerías necesarias (descomenta si es necesario en Colab)
#!pip install nltk scikit-learn

# 📚 Importar librerías
import nltk
import random
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 📥 Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')

# 🧠 Base de conocimiento
base_conocimiento = {
    "matemáticas": {
        "¿Qué es el álgebra?": "El álgebra es la rama de las matemáticas que estudia las estructuras, relaciones y cantidades usando letras y símbolos para representar números desconocidos.",
        "¿Cómo se resuelve una ecuación?": "Para resolver una ecuación, debes despejar la variable (la letra) realizando las mismas operaciones en ambos lados de la ecuación hasta que la variable quede sola.",
        "¿Qué es una función?": "Una función es una relación entre dos conjuntos donde cada elemento del primer conjunto se relaciona con exactamente un elemento del segundo conjunto.",
        "¿Para qué sirve la geometría?": "La geometría sirve para estudiar formas, tamaños, posiciones y propiedades del espacio. Es útil en arquitectura, diseño, arte y ingeniería.",
        "¿Qué son los números primos?": "Los números primos son números naturales mayores que 1 que solo son divisibles por 1 y por sí mismos. Ejemplo: 2, 3, 5, 7, 11, 13..."
    },

    "ciencias": {
        "¿Qué es la fotosíntesis?": "La fotosíntesis es el proceso por el cual las plantas convierten la luz solar, agua y dióxido de carbono en glucosa (azúcar) y oxígeno.",
        "¿Qué es el ADN?": "El ADN es como el 'manual de instrucciones' de todos los seres vivos. Contiene toda la información genética que determina nuestras características.",
        "¿Cómo funciona la gravedad?": "La gravedad es una fuerza invisible que atrae todos los objetos entre sí. Cuanto más masivo es un objeto, mayor es su atracción.",
        "¿Qué son los átomos?": "Los átomos son las partículas más pequeñas de la materia. Todo lo que nos rodea está hecho de átomos.",
        "¿Por qué el cielo es azul?": "El cielo es azul porque la luz azul se dispersa más que otros colores al entrar en la atmósfera."
    },

    "historia": {
        "¿Cuándo comenzó la Primera Guerra Mundial?": "La Primera Guerra Mundial comenzó en 1914 y terminó en 1918.",
        "¿Quién fue Simón Bolívar?": "Simón Bolívar fue un líder militar y político venezolano conocido como 'El Libertador'.",
        "¿Qué fue la Revolución Industrial?": "Fue un periodo de grandes cambios económicos y sociales (siglos XVIII-XIX) con la invención de máquinas.",
        "¿Cuándo llegó Colón a América?": "Cristóbal Colón llegó a América el 12 de octubre de 1492.",
        "¿Qué fueron las cruzadas?": "Fueron guerras religiosas en la Edad Media entre cristianos y musulmanes por Tierra Santa."
    },

    "literatura": {
        "¿Quién escribió Don Quijote?": "Miguel de Cervantes escribió 'Don Quijote de la Mancha'.",
        "¿Qué es una metáfora?": "Una metáfora es una figura retórica que consiste en identificar un término real con uno imaginario.",
        "¿Quién fue Shakespeare?": "William Shakespeare fue un dramaturgo y poeta inglés, autor de 'Romeo y Julieta' y 'Hamlet'.",
        "¿Qué es el modernismo?": "El modernismo fue un movimiento literario y artístico de finales del siglo XIX.",
        "¿Qué son los géneros literarios?": "Son formas de clasificar las obras: épico, lírico y dramático."
    }
}

# 🤝 Respuestas rápidas
respuestas_saludo = [
    "¡Hola! 😊 Soy tu chatbot escolar. ¿En qué materia te puedo ayudar?",
    "¡Buenos días! 🌟 Puedo responder sobre matemáticas, ciencias, historia y literatura.",
    "¡Hola! 👋 ¿Qué te gustaría aprender hoy?"
]

respuestas_despedida = [
    "¡Hasta pronto! 👋 Sigue aprendiendo.",
    "¡Nos vemos! 😊",
    "¡Adiós! 🌟 Que tengas un gran día."
]

# 🧹 Funciones de procesamiento de texto
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^\w\s¿?áéíóúñ]', ' ', texto)
    return ' '.join(texto.split())

def tokenizar_texto(texto):
    try:
        return nltk.word_tokenize(texto, language='spanish')
    except:
        return texto.split()

# 🎯 Funciones del chatbot
def detectar_saludo(pregunta):
    saludos = ['hola', 'buenos días', 'buenas tardes', 'buenas noches', 'saludos', 'qué tal', 'hey']
    return any(saludo in limpiar_texto(pregunta) for saludo in saludos)

def detectar_despedida(pregunta):
    despedidas = ['adiós', 'hasta luego', 'nos vemos', 'chau', 'bye', 'salir', 'terminar']
    return any(despedida in limpiar_texto(pregunta) for despedida in despedidas)

def calcular_similitud(pregunta_usuario, pregunta_base):
    try:
        vectorizador = CountVectorizer().fit([pregunta_usuario, pregunta_base])
        vectores = vectorizador.transform([pregunta_usuario, pregunta_base])
        return cosine_similarity(vectores[0], vectores[1])[0][0]
    except:
        palabras_usuario = set(tokenizar_texto(pregunta_usuario))
        palabras_base = set(tokenizar_texto(pregunta_base))
        return len(palabras_usuario & palabras_base) / max(len(palabras_usuario), len(palabras_base))

def encontrar_mejor_respuesta(pregunta_usuario):
    if detectar_saludo(pregunta_usuario):
        return random.choice(respuestas_saludo)
    if detectar_despedida(pregunta_usuario):
        return random.choice(respuestas_despedida)

    mejor_respuesta = ""
    mejor_puntuacion = 0
    materia_encontrada = ""

    for materia, preguntas in base_conocimiento.items():
        for pregunta_base, respuesta in preguntas.items():
            puntuacion = calcular_similitud(limpiar_texto(pregunta_usuario), limpiar_texto(pregunta_base))
            if puntuacion > mejor_puntuacion:
                mejor_puntuacion = puntuacion
                mejor_respuesta = respuesta
                materia_encontrada = materia

    if mejor_puntuacion > 0.2:
        return f"📚 [{materia_encontrada.title()}] {mejor_respuesta}"
    else:
        return random.choice([
            "🤔 No estoy seguro, ¿puedes reformular la pregunta?",
            "🔍 Intenta preguntarme sobre matemáticas, ciencias, historia o literatura.",
            "❓ Hmm, no tengo esa información."
        ])

# 💬 Interfaz principal
def iniciar_chatbot():
    print("🤖 ChatBot Escolar v1.0")
    print("Escribe 'salir' para terminar la conversación.\n")

    while True:
        usuario_input = input("🙋 Tú: ").strip()
        if not usuario_input:
            print("🤖 Bot: Por favor, escribe algo.")
            continue
        if detectar_despedida(usuario_input):
            print(f"🤖 Bot: {random.choice(respuestas_despedida)}")
            break
        respuesta = encontrar_mejor_respuesta(usuario_input)
        print(f"🤖 Bot: {respuesta}")

# 🚀 Iniciar chatbot
iniciar_chatbot()

